# Import openai library

In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
!git clone -q https://github.com/noahshinn/reflexion

In [3]:
%cd ./reflexion/hotpotqa_runs

/content/reflexion/hotpotqa_runs


In [4]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.9/770.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Init conversation

In [5]:
%cd ./reflexion/hotpotqa_runs

[Errno 2] No such file or directory: './reflexion/hotpotqa_runs'
/content/reflexion/hotpotqa_runs


In [6]:
import os
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [7]:
import sys, os
sys.path.append('..')
root = './root/'

In [8]:
from util import summarize_trial, log_trial, save_agents
from agents import CoTAgent, ReflexionStrategy

In [9]:
# download dataset
!gdown 1JpcJGLzldooWLCe2gPtlAlvRVXmwra8n

Downloading...
From: https://drive.google.com/uc?id=1JpcJGLzldooWLCe2gPtlAlvRVXmwra8n
To: /content/reflexion/hotpotqa_runs/QA_test.json
100% 26.3k/26.3k [00:00<00:00, 39.0MB/s]


In [11]:
import json
import random
n_sample = 20
data = json.load(open('QA_test.json', 'r'))
data = random.sample(data, n_sample)

In [12]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [13]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

In [15]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

agents = [CoTAgent(question = row['question'],
                   context = '',
                   key = row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples = COTQA_SIMPLE6,
                   reflect_prompt = cot_simple_reflect_prompt,
                   reflect_examples = COT_SIMPLE_REFLECTION,
                      ) for row in data]

In [16]:
n = 5
trial = 0
log = ''
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Thought: Let's think step by step. To solve this equation, we need to isolate the variable y. Subtract 4y from both sides to get -3 = y + 2. Then subtract 2 from both sides to get -5 = y.
Action: Finish[-5]
Answer: 4y - 3 = 5y + 2 => 4y - 5y = 2 + 3 => -y = 5 => y = -5.
Thought: Let's think step by step. Diện tích hình thang được tính bằng công thức: (đáy nhỏ + đáy lớn) * chiều cao / 2. Substituting the values, we get (4 + 6) * 5 / 2 = 10 * 5 / 2 = 50 / 2 = 25. So the answer is 25.
Action: Finish[25]
Answer: Diện tích hình thang = 1/2 x (đáy nhỏ + đáy lớn) x chiều cao = 1/2 x (4cm + 6cm) x 5cm = 25cm^2
Thought: Hãy tính diện tích của hình chữ nhật bằng cách nhân chiều dài và chiều rộng với nhau. Diện tích = 6cm x 4cm = 24cm².
Action: Finish[24cm²]
Answer: Diện tích hình chữ nhật = chiều dài x chiều rộng = 6cm x 4cm = 24cm^2
Thought: Hãy suy nghĩ từng bước. Số tự nhiên có 3 chữ số và chia hết cho 11, 13 và 17 chỉ có thể là 187. Kiểm tra bằng cách giảm 792 đơn vị từ 187, ta sẽ thấy rằng 

KeyboardInterrupt: 